In [1]:

from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import h5py
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import os


In [2]:
mypath = '/mnt/hdd/Mahsa/movies/Picking_your_brain/ananya_worms/Recording2023_07_20/results/temp' 

In [7]:
onlyfiles = [f for f in listdir(mypath)  if ".h5" in f]
for f in onlyfiles:
    h5fileAddress = os.path.join(mypath,f)
    h5 = h5py.File(h5fileAddress,'r+')
    convert_to_mask(h5)
    #change_fun(h5)
    h5.close()

Dataset name changed successfully.


In [3]:
def change_fun(h5):
    if 'pointdat' in h5.keys():
        del h5['pointdat'] 
    if 'helper_NN' in h5.keys():    
        h5.copy('helper_NN', 'pointdat')

In [6]:
def convert_to_mask(h5,neu=25):
    h5.attrs['pointdat'] = False #delete this so the gui open the file as new and set the parameters as point file
    if 'pointdat' in h5.keys():
        # Create a new dataset with the desired name ('pointdat_old')
        if 'pointdat_old' in h5.keys():
            del h5['pointdat_old']
        h5.copy('pointdat', 'pointdat_old')
        # Delete the old dataset ('pointdat')
        del h5['pointdat']
        print("Dataset name changed successfully.")
    else:
        print("Dataset 'pointdat' does not exist in the file.")
    h5.attrs['N_neurons'] = int(neu)

In [35]:
def convert_to_points(h5,neu=25):
    if 'pointdat' in h5.attrs.keys():
        del h5.attrs['pointdat'] #delete this so the gui open the file as new and set the parameters as point file
    if 'pointdat_old' in h5.keys():
        if 'pointdat' in h5.keys():
            del h5['pointdat']
        # Create a new dataset with the desired name ('pointdat_old')
        h5.copy('pointdat_old', 'pointdat')
        # Delete the old dataset ('pointdat')
        del h5['pointdat_old']
        print("Dataset name changed successfully.")
    else:
        print("Dataset 'pointdat_old' does not exist in the file.")
    h5.attrs['N_neurons'] = int(neu) #set this so the neuron bar shows up

In [7]:
GT=[5,10,210,310,410,510]
def Accept_frames(h5,GT):
    GT0 = h5['pointdat']
    prediction = h5['helper_NN']
    for i in GT:
        GT0[i,:,:] = prediction[i,:,:]
    del h5['pointdat'] 
    h5.create_dataset('pointdat',data=GT0)

In [37]:
hf=h5py.File('/mnt/hdd/Mahsa/movies/Picking_your_brain/ananya_worms/Recording2023_07_20/results/July14_121014_1_ctrl.h5','r+')

In [39]:
hx=h5py.File('/mnt/hdd/Mahsa/movies/Picking_your_brain/ananya_worms/Recording2023_07_20/results/NewResult_Alice/121014_1_ctrl.h5','r+')

In [40]:
print(np.shape(hx['1/frame']))
print(np.shape(hf['1/frame']))
hx['pointdat'] 

(1, 224, 160, 32)
(1, 224, 160, 32)


<HDF5 dataset "pointdat": shape (1582, 26, 3), type "<f4">

In [9]:
Accept_frames(hf,GT)

In [41]:
del hf['pointdat'] 
hf.create_dataset('pointdat',data=hx['helper_NN'] )

<HDF5 dataset "pointdat": shape (1582, 26, 3), type "<f8">

In [42]:
hf.close()
hx.close()